# Language Modeling

## Statistical Language Modeling

### Statistical Language Modeling, or Language Modeling and LM for short, is the development of probabilistic models that are able to predict the next word in the sequence given the words that precede it.


### The neural network approach to language modeling can be described using the three following model properties, taken from A Neural Probabilistic Language Model, 2003.

* Associate each word in the vocabulary with a distributed word feature vector.
* Express the joint probability function of word sequences in terms of the feature vectors of these words in the sequence.
* Learn simultaneously the word feature vector and the parameters of the probability function.


## How to Develop a Character-Based Neural Language Model

In [1]:
filename = '/Users/test/Documents/Software-projects/Python Projects/Deep-Learning-Projects/Deep-Learning-Overfitting-Cook-Book/data/rhyme.txt'

In [2]:

# load doc into memory
def load_doc(filename):
  # open the file as read only 
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file 
  file.close()
  return text

# save tokens to file, one dialog per line
def save_doc(lines, filename): 
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()

# load text
raw_text = load_doc(filename) 
print(raw_text)
# clean
tokens = raw_text.split() 
raw_text = ' '.join(tokens)
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
  # select sequence of tokens
  seq = raw_text[i-length:i+1]
  # store
  sequences.append(seq)
print('Total Sequences: %d' % len(sequences)) # save sequences to file
out_filename = 'char_sequences.txt' 
save_doc(sequences, out_filename)


Sing a song of sixpence,
A pocket full of rye.
Four and twenty blackbirds,
Baked in a pie.
When the pie was opened The birds began to sing; Wasn't that a dainty dish, To set before the king.
The king was in his counting house,
Counting out his money;
The queen was in the parlour,
Eating bread and honey.
The maid was in the garden,
Hanging out the clothes,
When down came a blackbird
And pecked off her nose.
Total Sequences: 399
